In [1]:
from examples.util.getDataset import getDataset
from examples.util.plotting import plot_w
from examples.util.test_fns import *
from examples.util.testing import shrimp_v_random, growth
from examples.util.metrics import shrimp_test, make_l2_loss
from sparse_rf.util import *
from sparse_rf.modules import make_A, make_W
from sparse_rf.algs import shrimp, l2, l1, sindy
from functools import partial
import matplotlib.pyplot as plt
from math import ceil
import gc

In [3]:
r1 = 1
r2 = 2
r3 = 3
r4 = 4
fn = partial(fn8, r1=r1, r2=r2, r3=r3, r4=r4)
q = 2
d = 10
m = 200
N = 1500
l = 1e-13

Xtr, Xte, Ytr, Yte = getDataset(fn, d=d, m=m)

trials = 10

scales = np.arange(1/q, 206, 5)

In [4]:
for scale_ in scales:
    svr_q = shrimp_v_random(trials, Xtr, Ytr, Xte, Yte, q, N, l, scale=np.sqrt(scale_))
    errs_bp = []
    for _ in range(trials):
        W, _ = make_W(d, q, N, scale=1/np.sqrt(scale_))
        Atr = make_A(Xtr, W)
        Ate = make_A(Xte, W)
        c = l1(Atr, Ytr)
        Ypred = Ate@c
        err_bp = np.sum((Ypred-Yte)**2)/len(Yte)

        errs_bp.append(err_bp)

        del W
        del Atr
        del Ate
        del c
        del Ypred

        gc.collect()

    print("Variance: {}".format(scale_))
    print("Min l2: {}".format(svr_q.err_l2))
    print("SHRIMP: {}".format(svr_q.err_shrimp))
    print("Random pruning: {}".format(svr_q.err_r))
    print("Min l1: {}".format(np.mean(errs_bp)))
    print("----------------------------")

    del errs_bp
    del svr_q

    gc.collect()

Variance: 0.5
Min l2: 23.855150044986953
SHRIMP: 4.207830305565635
Random pruning: 4.454843220465175
Min l1: 6.040959632871117
----------------------------
Variance: 5.5
Min l2: 4.840322566888486
SHRIMP: 0.9525515031212134
Random pruning: 4.333917495366255
Min l1: 8.589743626617144
----------------------------
Variance: 10.5
Min l2: 4.305264824720575
SHRIMP: 0.6043619105743938
Random pruning: 4.4902066222337265
Min l1: 7.478520767722442
----------------------------
Variance: 15.5
Min l2: 4.685954367629127
SHRIMP: 0.31770187444269726
Random pruning: 4.501444866416624
Min l1: 6.348756123175393
----------------------------
Variance: 20.5
Min l2: 4.322487868256277
SHRIMP: 0.5698360615561888
Random pruning: 4.332620703805016
Min l1: 5.6865015896610345
----------------------------
Variance: 25.5
Min l2: 4.498790549753744
SHRIMP: 0.5568850997772066
Random pruning: 5.022047470282362
Min l1: 5.4696367842652505
----------------------------
Variance: 30.5
Min l2: 4.638828920282838
SHRIMP: 1.10330